In [ ]:
import numpy as np
import polars as pl

In [ ]:
import gzip
import shutil
import os
from tqdm import tqdm

In [ ]:
from constants import *

In [ ]:
corpus_columns = ["entity1", "description"]
corpus_text = pl.read_csv(f"datasets/{DatasetNames.corpus.value}", sep="\t", has_header=False, new_columns=corpus_columns, encoding="utf8-lossy")

In [ ]:
corpus_text = corpus_text.sort(by="entity1")
entity_paragraphs = corpus_text.select("description").to_series()

## Sentence transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

In [ ]:
corpus_text.write_csv("descriptions.csv")

In [ ]:
import pandas as pd

df = pd.read_csv("descriptions.csv", encoding="latin_1")

In [ ]:
entity_paragraphs = df["description"]

In [ ]:
sentences = list(entity_paragraphs)

In [ ]:
all_embeddings = []
step = 500_000

for cycle in range(0, len(sentences), step):
    embeddings = model.encode(sentences[cycle:cycle+step], show_progress_bar=True)
    all_embeddings.append(embeddings)

In [ ]:
embeddings = np.concatenate(all_embeddings)

In [ ]:
embeddings.nbytes

In [ ]:
np.save("sentence_features", embeddings)

In [ ]:
embeddings.shape